The returned data structure should include:

* the total price for the order
* the total VAT for the order
* price and VAT for each item in the order

Load example order and pricing dicts

In [1]:
# provided
raw_pricing = {
	"prices": [
		{
			"product_id": 1,
			"price": 599,
			"vat_band": "standard"
		},
		{
			"product_id": 2,
			"price": 250,
			"vat_band": "zero"
		},
		{
			"product_id": 3,
			"price": 250,
			"vat_band": "zero"
		},
		{
			"product_id": 4,
			"price": 1000,
			"vat_band": "zero"
		},
		{
			"product_id": 5,
			"price": 1250,
			"vat_band": "standard"
		}
	],
	"vat_bands": {
		"standard": 0.2,
		"zero": 0
	}
}

format pricing into a dict where key is product_id

In [2]:
def format_pricing(pricing : dict):
    return {product["product_id"]:{"price":product["price"], 
                                    "VAT":round(pricing["vat_bands"][product["vat_band"]]*product["price"], 2) # round to nearest penny
                                  } for product in pricing["prices"]}

In [3]:
pricing = format_pricing(raw_pricing)
pricing

{1: {'price': 599, 'VAT': 119.8},
 2: {'price': 250, 'VAT': 0},
 3: {'price': 250, 'VAT': 0},
 4: {'price': 1000, 'VAT': 0},
 5: {'price': 1250, 'VAT': 250.0}}

In [4]:
# Provided
order = {
    "order": {
        "id": 12345,
        "currency": "GBP", # optional
        "customer": {
            ...
        },
        "items": [
            {
                "product_id": 1,
                "quantity": 1
            },
            {
                "product_id": 2,
                "quantity": 5
            },
            {
                "product_id": 3,
                "quantity": 1
            }
        ]
    }
}

Format order into a dict where the key is the product_id and value is the quantity

In [5]:
def format_order(order : dict):
    return {item["product_id"]:item["quantity"] for item in order["order"]["items"]}

order_quantities = format_order(order)
order_quantities

{1: 1, 2: 5, 3: 1}

We can now process the order

In [6]:
def calc_order_details(order_quantities : dict, pricing : dict):
    summary = {}
    running_total_price = 0
    running_total_vat = 0
    
    for product_id in order_quantities:
        summary[product_id] = {}
        quantity = order_quantities[product_id] 
        
        total_price = quantity * pricing[product_id]['price']
        summary[product_id]["total_price"] = total_price
        running_total_price += total_price
        
        total_vat = quantity * pricing[product_id]['VAT']
        summary[product_id]["total_VAT"] = total_vat
        running_total_vat += total_vat
    
    summary["total_price"] = running_total_price
    summary["total_VAT"] = running_total_vat
    return summary

In [7]:
order_details = calc_order_details(order_quantities, pricing)

In [8]:
order_details

{1: {'total_price': 599, 'total_VAT': 119.8},
 2: {'total_price': 1250, 'total_VAT': 0},
 3: {'total_price': 250, 'total_VAT': 0},
 'total_price': 2099,
 'total_VAT': 119.8}

In [10]:
# Handle currency - need to apply a conversion rate
currency = None

if not "currency" in order["order"].keys():
    currency = "GBP"
else:
    currency = order["order"]["currency"]
    
currency

'GBP'